In [1]:
## Importing necessary Libraries 
import ctypes, inspect, os, graphlab
from graphlab import SFrame
import pandas as pd

In [2]:
# This code reads the data into a graphlab Sframe. The dataset used is an Office Products dataset containing 53,257 reviews   
# retrieved from: http://jmcauley.ucsd.edu/data/amazon/ citation: "Image-based recommendations on styles and 
# substitutes" J. McAuley, C. Targett, J. Shi, A. van den Hengel, SIGIR, 2015
Office_Products = graphlab.SFrame.read_json('Office_Products_5.json', orient='lines')

2017-04-08 21:46:08,519 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\Chauncey\AppData\Local\Temp\graphlab_server_1491702367.log.0


Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 100 lines in 0.985056 secs.

This non-commercial license of GraphLab Create for academic use is assigned to raggiechauncey@gmail.com and will expire on April 23, 2017.
------------------------------------------------------

Read 50429 lines. Lines per second: 29198.7

Finished parsing file C:\Users\Chauncey\Office_Products_5.json

Parsing completed. Parsed 53258 lines in 1.7591 secs.


Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
## Exploring the dataset
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [4]:
## Counting the number of rows 
len(Office_Products)

53258

In [20]:
## Checking the rating distribution. Most reviews have a 5 rating.
Office_Products['overall'].show(view='Categorical') 

Canvas is updated and available in a tab in the default browser.


In [18]:
## Checking the rating distribution. Most reviews have a 5 rating. 30,327 votes (56.944%)
Office_Products['overall'].show(view='Categorical') 

In [6]:
# building a word count vector for each review 
Office_Products['word_count'] = graphlab.text_analytics.count_words(Office_Products['reviewText'])

In [8]:
Office_Products['word_count'].head()

dtype: dict
Rows: 10
[{'area.': 1L, 'hand,': 1L, 'over': 1L, 'move': 1L, 'aside': 1L, 'batteries': 1L, 'decade': 1L, '12c,': 2L, 'powerful,': 1L, 'counterintuitive...and': 1L, 'to': 7L, 'other': 1L, '(now': 1L, 'mistake!': 1L, 'it,': 1L, 'good': 1L, '49g': 3L, 'know': 1L, 'using': 1L, 'one': 2L, 'best.': 1L, "order.i'm": 1L, 'lost': 1L, 'manual': 2L, 'me.if': 1L, 'served': 1L, 'view': 1L, 'set': 1L, 'old': 1L, 'people': 1L, 'back': 1L, 'decided': 1L, 'are': 1L, 'even': 1L, 'will': 1L, 'what': 2L, 'for': 4L, 'calculator,': 1L, 'looking': 1L, 'does': 2L, 'travelling.': 1L, 'bought': 1L, 'be': 3L, 'business': 1L, 'found': 1L, 'come': 1L, 'by': 1L, 'on': 1L, 'about': 1L, 'of': 2L, 'sketchy': 1L, "it's": 1L, "i'm": 1L, 'programmable.': 1L, 'or': 1L, 'first': 1L, 'too.': 1L, 'sense;': 1L, 'replace': 2L, 'another': 1L, '1984': 1L, 'statistics': 1L, '49g.': 1L, 'faithfully': 1L, 'home!': 1L, 'way': 1L, 'basic': 1L, 'was': 2L, 'until': 1L, 'you.': 1L, 'buy': 1L, 'that': 3L, 'hp': 3L, 'so,': 2L,

In [14]:
# Limiting the review data to only helpful reviews. The ‘helpful’ column contains values like ‘[x, y]’.
# The first value represents the number of helpful votes, the second represents overall votes. 
# If at least 60% consider the review helpful, we want those reviews. A binary 1 or 0 column will identify the review as helpful or not.
import numpy as np

Office_Products['helpful_count'] = Office_Products['helpful'].apply(lambda x: x[0])
Office_Products['overall_count'] = Office_Products['helpful'].apply(lambda x: x[1])
Office_Products['helpful_percent'] = Office_Products['helpful_count'] / Office_Products['overall_count']
Office_Products['helpful_review'] = np.where((Office_Products['helpful_percent'] >= .6) & (Office_Products['overall_count'] > 0), 1, 0)


In [15]:
Office_Products.head()

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName
B00000JBLH,"[3, 4]",5.0,"I bought my first HP12Cin about 1984 or so, and ...","09 3, 2004",A32T2H8150OJLU,ARH
B00000JBLH,"[7, 9]",5.0,WHY THIS BELATED REVIEW?I feel very obliged to ...,"12 15, 2007",A3MAFS04ZABRGO,"Let it Be ""Alan"""
B00000JBLH,"[3, 3]",2.0,I have an HP 48GX thathas been kicking for ...,"01 1, 2011",A1F1A0QQP2XVH5,Mark B
B00000JBLH,"[7, 8]",5.0,I've started doing morefinance stuff recently ...,"04 19, 2006",A49R5DBXXQDE5,R. D Johnson
B00000JBLH,"[0, 0]",5.0,For simple calculationsand discounted cash ...,"08 4, 2013",A2XRMQA6PJ5ZJ8,Roger J. Buffington
B00000JBLH,"[10, 12]",5.0,"While I don't have anMBA, it's hard to bel ...","01 23, 2002",A2JFOHC9W629IE,scott_from_dallas
B00000JBLH,"[3, 4]",5.0,I've had an HP 12C eversince they were first ...,"01 17, 2007",A38NELQT98S4H8,W. B. Halper
B00000JBLH,"[0, 0]",5.0,Bought this for my bossbecause he lost his. He ...,"11 14, 2013",AA8M6331NI1EN,ZombieMom
B00000JBLU,"[3, 3]",5.0,"This is a well-designed,simple calculator that ...","12 7, 2010",A25C2M3QF9G7OQ,Comdet
B00000JBLU,"[0, 0]",5.0,"I love this calculator,big numbers and calcu ...","12 2, 2013",A1RTVWTWZSIC94,"Hb ""Black Beauty"""


In [16]:
# Building the sentiment classifier. Performing a random 80/20 split to select the training and testing data.
train_data, test_data = Office_Products.random_split(.8, seed=0)

In [17]:
sentiment_model_Logist = graphlab.logistic_classifier.create(train_data,
                                                            target = 'helpful_review',
                                                            features = ['word_count'],
                                                            validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 42649

Number of classes           : 2

...

| 6         | 10       | 1.000000  | 6.207355     | 0.971465          | 0.718447            |

| 10        | 14       | 1.000000  | 9.226528     | 0.987268          | 0.684419            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [18]:
## evaluating the model
sentiment_model_Logist.evaluate(test_data, metric = 'roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+------+------+
 | threshold |      fpr       |      tpr       |  p   |  n   |
 +-----------+----------------+----------------+------+------+
 |    0.0    |      1.0       |      1.0       | 2907 | 7702 |
 |   1e-05   | 0.987795377824 | 0.986240110079 | 2907 | 7702 |
 |   2e-05   | 0.983640612828 | 0.981080151359 | 2907 | 7702 |
 |   3e-05   | 0.979745520644 | 0.978672170623 | 2907 | 7702 |
 |   4e-05   | 0.976239937679 | 0.974888200894 | 2907 | 7702 |
 |   5e-05   | 0.974032718774 | 0.972824217406 | 2907 | 7702 |
 |   6e-05   | 0.972085172682 | 0.97041623667  | 2907 | 7702 |
 |   7e-05   | 0.970786808621 | 0.969384244926 | 2907 | 7702 |
 |   8e-05   | 0.968839262529 | 0.968008255934 | 2907 | 7702 |
 |   9e-05   | 0.96715138925  | 0.966632266942 | 2907 | 7702 |
 +-----------+----------------+----------------+------+------+
 [100001 row

In [19]:
# True Positive = 1189 False Negative = 1718 Accuracy = 0.685 Precision = 0.423 false Positive = 1625 True Negative = 6077 recall = 0.409
sentiment_model_Logist.show(view = 'Evaluation')

Canvas is updated and available in a tab in the default browser.


In [25]:
## Trying a SVM model
sentiment_model_SVM = graphlab.svm_classifier.create(train_data,
                                                     target = 'helpful_review',
                                                     features = ['word_count'],
                                                     validation_set = test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

SVM:

--------------------------------------------------------

Number of examples          : 42649

Number of classes           : 2

...

| 6         | 11       | 1.000000  | 5.802332     | 0.967080          | 0.720615            |

| 10        | 15       | 1.000000  | 8.420481     | 0.980891          | 0.689886            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [27]:
## Evaluating the model
coefficients = sentiment_model_SVM['coefficients'] 

In [28]:
# Make predictions (as margins, or class)
predictions = sentiment_model_SVM.predict(Office_Products)    # Predicts 0/1
predictions = sentiment_model_SVM.predict(Office_Products, output_type='margin')

In [29]:
results = sentiment_model_SVM.evaluate(Office_Products)  

In [33]:
results

{'accuracy': 0.9229223778587254, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  2658 |
 |      0       |        1        |  1447 |
 |      0       |        0        | 36779 |
 |      1       |        1        | 12374 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.8577270994350674, 'precision': 0.8953042471601187, 'recall': 0.8231772219265566}

In [35]:
Office_Products['Predicted_Helpfulness'] = sentiment_model_SVM.predict(sentiment_model_SVM, output_type = 'margin')

ToolkitError: Input dataset is not an SFrame. If it is a Pandas DataFrame, you may use the to_sframe() function to convert it to an SFrame.